In [1]:
#Leader code bot 1.12

In [ ]:
import paho.mqtt.client as mqtt
import time
import os
import threading

# MQTT settings
BROKER = "158.39.162.129"
PORT = 1883

ROLE = "leader"  # Define this JetBot as the leader
LEADER_IP = os.getenv("LEADER_IP", "192.168.1.186")  # Default IP for leader

if ROLE == "leader":
    print(f"This JetBot is the Leader with IP: {LEADER_IP}")
elif ROLE == "follower":
    print(f"This JetBot is a Follower. Connecting to Leader at IP: {LEADER_IP}")
else:
    print("Error: Role not defined properly")
    
TOPIC_LEADER = "platoon/leader"
TOPIC_FOLLOWERS = "platoon/followers"
TOPIC_STATUS = "platoon/status"
TOPIC_SYNC = "platoon/sync"

# Initialize the MQTT client
client = mqtt.Client()

def on_connect(client, userdata, flags, rc):
    print("Connected with result code", rc)
    client.subscribe(TOPIC_FOLLOWERS)  # Feedback from followers
    client.subscribe(TOPIC_STATUS)    # Followers' status
    client.publish(TOPIC_LEADER, "move", retain=True)
    
def on_disconnect(client, userdata, rc):
    print("Disconnected. Attempting to reconnect...")
    while True:
        try:
            client.reconnect()
            break
        except:
            time.sleep(1)
client.on_disconnect = on_disconnect

def on_message(client, userdata, msg):
    print(f"Received on {msg.topic}: {msg.payload.decode()}")
    
def send_heartbeat():
    while True:
        client.publish("platoon/heartbeat", "leader_online")
        time.sleep(10)  # Adjust frequency as needed

threading.Thread(target=send_heartbeat, daemon=True).start()


# Setup callbacks
client.on_connect = on_connect
client.on_message = on_message

# Connect to the broker
client.connect(BROKER, PORT, 60)

# Start the loop
client.loop_start()

# Command and synchronization loop
try:
    while True:
        command = input("Enter command for followers (e.g., 'move', 'stop', 'move right' , 'move left', 'angle'): ")
        client.publish(TOPIC_LEADER, command)
        print(f"Command '{command}' sent to followers.")

        sync_signal = input("Send sync signal (e.g., 'sync', 'align') or leave blank: ")
        if sync_signal:
            client.publish(TOPIC_SYNC, sync_signal)
            print(f"Sync signal '{sync_signal}' sent to followers.")

        time.sleep(1)
except KeyboardInterrupt:
    print("Exiting...")
finally:
    client.loop_stop()
    client.disconnect()
    